# Absorption correction for cylindrical samples

The intensity at a detector element located at $\mathbf{x}$ is modeled as
$$
I(\lambda, \mathbf{x}) = \int_{sample} S(\lambda, \arccos\big(\frac{\hat{\mathbf{b}} \cdot (\mathbf{x}-\mathbf{x}')}{||\mathbf{x}-\mathbf{x}'||}\big)) T(\lambda, \mathbf{x}, \mathbf{x}') \ d\mathbf{x}'
$$
where $S(\lambda, \theta)$ is the probability that a neutron with wavelength $\lambda$ scatters with the scattering angle $\theta$ and $T(\lambda, \mathbf{x}, \mathbf{x}')$ is the transmission rate for neutrons scattering at $\mathbf{x}'$ towards $\mathbf{x}$, and $\hat{\mathbf{b}}$ is the incoming beam direction.

If the detector is far away from the sample relative to the size of the sample, then $\frac{\hat{\mathbf{b}} \cdot (\mathbf{x}-\mathbf{x}')}{||\mathbf{x}-\mathbf{x}'||}$ is close to constant and the $S$ term can be moved outside of the integtral:
$$
I(\lambda, \mathbf{x}) \approx S(\lambda, \theta(\mathbf{x})) \int_{sample}  T(\lambda, \mathbf{x}, \mathbf{x}') \ d\mathbf{x}'.
$$
To compute the scattering probabiltiy $S$ from the intensity $I$ we need to estimate the term
$$
C(\lambda, \mathbf{x}) = \int_{sample}  T(\lambda, \mathbf{x}, \mathbf{x}') \ d\mathbf{x}'.
$$
This is the "absorption correction" term.

The transmission fraction is a function of path length $L$ of the neutron going through the sample
$$
T(\lambda, \mathbf{x}, \mathbf{x}') = \exp{(-\mu(\lambda) L(\mathbf{x}, \mathbf{x}'))}
$$
where $\mu$ is material dependent.

The path length through the sample depends on the shape of the sample, the scattering point $\mathbf{x}'$ and the detection position $\mathbf{x}$.

To compute $C(\lambda, \mathbf{x})$ you can use
```python
scippneutron.absorption.base.compute_transmission_map(
    shape,
    material,
    beam_direction,
    wavelength,
    detector_position
)
```
where `shape` and `material` are sample properties, `beam_direction` corresponds to $\mathbf{b}$, `wavelength` corresponds to $\lambda$ and `detector_position` corresponds to $\mathbf{x}$. 

In [ ]:
import scipp as sc
import numpy as np

from scippneutron.absorption import compute_transmission_map
from scippneutron.absorption.cylinder import Cylinder


class DummyMaterial:
    'This is probably not a good physical model, but it illustrates the interface'
    def absorption_coefficient(wavelength):
        return sc.scalar(1, unit='1/(cm * angstrom)') * wavelength


sample_shape = Cylinder(
    symmetry_line=sc.vector([0, 1, 0]),
    center_of_base=sc.vector([0, -2, 0], unit='cm'),
    radius=sc.scalar(1, unit='cm'),
    height=sc.scalar(4, unit='cm')
)

# Create some dummy detector positions.

## Note
## If the detector array has a large number of elements
## it's best for performance to not evaluate the correction for every detector element
## but instead to evaluate it over some grid of positions and
## interpolate to the positions of the detector elements.
theta = sc.linspace('theta', 0, np.pi, 100, endpoint=True, unit='rad')
phi = sc.linspace('phi', 0, 2 * np.pi, 200, endpoint=False, unit='rad')
r = sc.array(dims='r', values=[2.], unit='m')
dims = (*r.dims, *theta.dims, *phi.dims)

# Detector positions in grid on a sphere around the origin
detector_positions = sc.vectors(
    dims=dims,
    values=sc.concat(
        [
            r * sc.sin(theta) * sc.cos(phi),
            r * sc.sin(theta) * sc.sin(phi),
            sc.broadcast(r * sc.cos(theta), sizes={**r.sizes, **theta.sizes, **phi.sizes}),
        ],
        dim='row',
    )
    .transpose([*dims, 'row'])
    .values,
    unit=r.unit,
)

def transmission(quadrature_kind):
    'Evaluates the transmission map for different kinds of quadrature'
    da = compute_transmission_map(
        sample_shape,
        DummyMaterial,
        # Assume beam goes in z-direction
        beam_direction=sc.vector([0, 0, 1]),
        wavelength=sc.linspace('wavelength', 0.5, 2.5, 10, unit='angstrom'),
        detector_position=detector_positions,
        quadrature_kind=quadrature_kind,
    )
    da.coords['phi'] = phi
    da.coords['theta'] = theta
    return da


def show_correction_map(da):
    'Plotting utility'
    return (
        da['wavelength', 0]['r', 0].plot() /
        da['wavelength', 0]['r', 0]['theta', da.sizes['theta']//2].plot() /
        da['wavelength', 0]['r', 0]['theta', da.sizes['theta']//2]['phi', da.sizes['phi']//4 - da.sizes['phi']//6:da.sizes['phi']//4 + da.sizes['phi']//6].plot() /
        da['wavelength', 0]['r', 0]['theta', da.sizes['theta']//2]['phi', da.sizes['phi']//2 - da.sizes['phi']//6:da.sizes['phi']//2 + da.sizes['phi']//6].plot() /
        da['wavelength', 0]['r', 0]['phi', da.sizes['phi']//2].plot()
    )

In [ ]:
transmission('cheap')

In [ ]:
show_correction_map(transmission('cheap'))

In [ ]:
show_correction_map(transmission('medium'))

In [ ]:
show_correction_map(transmission('expensive'))

## Error estimate

In [ ]:
import plopp as pp
cheap = transmission('cheap')['wavelength', 0]['r', 0]
medium = transmission('medium')['wavelength', 0]['r', 0]
expensive = transmission('expensive')['wavelength', 0]['r', 0]

t = cheap.sizes['theta'] // 2
pmin = cheap.sizes['phi'] // 2 - cheap.sizes['phi'] // 8
pmax = cheap.sizes['phi'] // 2 + cheap.sizes['phi'] // 8
pp.plot(
    {
        'cheap': cheap['theta', t]['phi', pmin:pmax],
        'medium': medium['theta', t]['phi', pmin:pmax],
        'expensive': expensive['theta', t]['phi', pmin:pmax],
    }
)

In [ ]:
# Relative difference between cheap and expensive quadrature
(sc.abs(cheap - expensive)/sc.abs(expensive)).mean().value

In [ ]:
# Relative difference between medium and expensive quadrature
(sc.abs(medium - expensive)/sc.abs(expensive)).mean().value